#### RAG Pipelines - Data Ingestion to Vector DB Pipeline

In [19]:
import os
from langchain_community.document_loaders import PyMuPDFLoader,PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [20]:
### Read all the pdf's inside the directory

def process_all_pdfs(pdf_directory):
    """"Process all PDF files in a directory"""
    all_documents=[]
    pdf_dir=Path(pdf_directory)

    ### Find all PDF files recursively
    pdf_files=list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\n Processing:{pdf_file.name}")
        try:
            loader=PyPDFLoader(str(pdf_file))
            documents=loader.load()

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file']=pdf_file.name
                doc.metadata['file_type']='pdf'
            
            all_documents.extend(documents)
            print(f" Loaded {len(documents)} pages")
        except Exception as e:
            print(f" Error:{e}")
    print(f"\n Total documents loaded:{len(all_documents)}")
    return all_documents     

# Process all PDFs in the data directory

all_pdf_documents=process_all_pdfs("../data")

Found 4 PDF files to process

 Processing:Cross Validation.pdf
 Loaded 12 pages

 Processing:Data Modelling.pdf
 Loaded 19 pages

 Processing:DBMS.pdf
 Loaded 7 pages

 Processing:ML Notes.pdf
 Loaded 47 pages

 Total documents loaded:85


In [21]:
all_pdf_documents

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-01-30T16:08:48+00:00', 'title': 'Untitled', 'author': 'Lakshmanan Kathiresan', 'moddate': '2024-01-30T16:08:48+00:00', 'source': '..\\data\\PDF\\Cross Validation.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1', 'source_file': 'Cross Validation.pdf', 'file_type': 'pdf'}, page_content='lOMoARcPSD|285 747 87 \n \n \n \n \nUNIT V       DESIGN AND ANALYSIS OF MACHINE LEARNING EXPERIMENTS         8 \nGuidelines for machine learning experiments, Cross Validation (CV) and resampling – K- \nfold CV, bootstrapping, measuring classifier performance, assessing a single classification \nalgorithm and comparing two classification algorithms – t test, Mc Nemar’s test, K -fold \nCV paired t test \n \n5.1 Guidelines for Machine Learning Experiments \n \nThe steps in machine learning are the same as for any type of \nexperimentation, that at this point, it is  not important whether the task

In [22]:
#### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n","\n"," ", ""]
    )
    split_docs=text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example of a chunk
    if split_docs:
        print(f"\nExample Chunk:")
        print(f"Content:{split_docs[0].page_content[:200]}...")
        print(f"Metadata:{split_docs[0].metadata}")

    return split_docs    

In [23]:
chunks=split_documents(all_pdf_documents)
chunks

Split 85 documents into 146 chunks

Example Chunk:
Content:lOMoARcPSD|285 747 87 
 
 
 
 
UNIT V       DESIGN AND ANALYSIS OF MACHINE LEARNING EXPERIMENTS         8 
Guidelines for machine learning experiments, Cross Validation (CV) and resampling – K- 
fold ...
Metadata:{'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-01-30T16:08:48+00:00', 'title': 'Untitled', 'author': 'Lakshmanan Kathiresan', 'moddate': '2024-01-30T16:08:48+00:00', 'source': '..\\data\\PDF\\Cross Validation.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1', 'source_file': 'Cross Validation.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-01-30T16:08:48+00:00', 'title': 'Untitled', 'author': 'Lakshmanan Kathiresan', 'moddate': '2024-01-30T16:08:48+00:00', 'source': '..\\data\\PDF\\Cross Validation.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1', 'source_file': 'Cross Validation.pdf', 'file_type': 'pdf'}, page_content='lOMoARcPSD|285 747 87 \n \n \n \n \nUNIT V       DESIGN AND ANALYSIS OF MACHINE LEARNING EXPERIMENTS         8 \nGuidelines for machine learning experiments, Cross Validation (CV) and resampling – K- \nfold CV, bootstrapping, measuring classifier performance, assessing a single classification \nalgorithm and comparing two classification algorithms – t test, Mc Nemar’s test, K -fold \nCV paired t test \n \n5.1 Guidelines for Machine Learning Experiments \n \nThe steps in machine learning are the same as for any type of \nexperimentation, that at this point, it is  not important whether the task

### Embedding and VectorDB


In [24]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self,model_name:str="all-MiniLM-L6-v2"):
        """
        Initalize the embedding manager

        Args:
            model_name="HuggingFace model name for sentence embeddings
        """

        self.model_name=model_name
        self.model=None
        self.load_model()

    def load_model(self):
        """Load the SentenceTransformer model """
        try:
            print(f"Loading embedding model:{self.model_name}")
            self.model=SentenceTransformer(self.model_name)
            print(f"Model loaded succesfully. Embedding dimension:{self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model{self.model_name}:{e}")
            raise    
    def generate_embeddings(self,texts:List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
           texts:List of text strings to embed
        Returns:
           numpy array of embeddings with shape(len(texts),embedding_dim)

        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings=self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape:{embeddings.shape}")
        return embeddings
    

### initalize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model:all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 312.34it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded succesfully. Embedding dimension:384


### Vector Store


In [26]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self,collection_name:str="pdf_documents",persist_directory:str="../data/vector_store"):
        """
        Initialize the vector store

        Args:
           collection_name:Name of the ChromaDB collection
           persist_directory: Directory to persist the vector store
        """
        self.collection_name=collection_name
        self.persist_directory=persist_directory
        self.client=None
        self.collection=None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB Client
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client=chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection=self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description":"PDF document embeddings for RAG"}

            )    
            print(f"Vector store initialized. Collection:{self.collection_name}")
            print(f"Existing documents in collection:{self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store:{e}")
            raise   

    def add_documents(self,documents:List[Any],embeddings:np.ndarray):
        """
        Add documents and their embedings to the vector stor
        
        Args:
            documents= List of Langchain documents
            embeddings= Corresponding embeddings fro the documents                                                                                                                        
            
        """
        if len(documents)!=len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        print(f"Adding {len(documents)} documents to vector store...")

        # prepare data for ChromaDB
        ids=[]
        metadatas=[]
        documents_text=[]
        embeddings_list=[]

        for i,(doc,embeddings) in enumerate(zip(documents,embeddings)):
            # Generate Unique ID
            doc_id=f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            #prepare metadata
            metadata=dict(doc.metadata)
            metadata['doc_index']=i
            metadata['content_length']=len(doc.page_content)
            metadatas.append(metadata)

            #Documents content
            documents_text.append(doc.page_content)

            #Embedding
            embeddings_list.append(embeddings.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )   
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection:{self.collection.count()}")
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise     

vectorstore=VectorStore()
vectorstore



Vector store initialized. Collection:pdf_documents
Existing documents in collection:438


In [27]:
chunks

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-01-30T16:08:48+00:00', 'title': 'Untitled', 'author': 'Lakshmanan Kathiresan', 'moddate': '2024-01-30T16:08:48+00:00', 'source': '..\\data\\PDF\\Cross Validation.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1', 'source_file': 'Cross Validation.pdf', 'file_type': 'pdf'}, page_content='lOMoARcPSD|285 747 87 \n \n \n \n \nUNIT V       DESIGN AND ANALYSIS OF MACHINE LEARNING EXPERIMENTS         8 \nGuidelines for machine learning experiments, Cross Validation (CV) and resampling – K- \nfold CV, bootstrapping, measuring classifier performance, assessing a single classification \nalgorithm and comparing two classification algorithms – t test, Mc Nemar’s test, K -fold \nCV paired t test \n \n5.1 Guidelines for Machine Learning Experiments \n \nThe steps in machine learning are the same as for any type of \nexperimentation, that at this point, it is  not important whether the task

In [28]:
### Convert the text to embedding
texts=[doc.page_content for doc in chunks ]

## Generate the embeddings

embeddings=embedding_manager.generate_embeddings(texts)

## Store in the vector database

vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 146 texts...


Batches: 100%|██████████| 5/5 [00:08<00:00,  1.67s/it]


Generated embeddings with shape:(146, 384)
Adding 146 documents to vector store...
Successfully added 146 documents to vector store
Total documents in collection:584


#### Retrieve Pipeline from VectorStore

In [29]:
class RAGRetriever:
    """Handles query based retrieval from the vetcor stores"""
    def __init__(self,vector_store:VectorStore,embedding_manager=EmbeddingManager):
        """
        Initialize the retriever

        Args:
           vetcor_store: Vector store containing document embeddings
           embedding_manager: Manager for generating query embeddings
        """

        self.vector_store=vector_store
        self.embedding_manager=embedding_manager
        
    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)


In [30]:
rag_retriever

In [31]:
rag_retriever.retrieve("What is machine learning")

Retrieving documents for query: 'What is machine learning'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 71.40it/s]

Generated embeddings with shape:(1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_0a6c28a6_92',
  'content': 'Definition of machine learning \nArthur Samuel, an early American leader in the field of  computer gaming and \nartificial intelligence, coined the  term “Machine Learning”  in 1959  while at \nIBM. \nHe defined machine learning as “the field of study that gives computers the \nability to learn  without being explicitly programmed.” However, there is no \nuniversally accepted definition  for machine learning. Different authors define \nthe term differently. We give below two more definitions. \n1. Machine learning is programming computers to optimize a performance \ncriterion using example data or past experience. We have a model defined up to \nsome parameters, and learning is the execution of a computer program to \noptimize the parameters of the model using the  training data or past experience. \nThe model may be predictive to make predictions in the future, or descriptive to \ngain knowledge from data, or both. \n2. The field of study known

In [32]:
rag_retriever.retrieve("What is K-Fold Cross-Validation ")

Retrieving documents for query: 'What is K-Fold Cross-Validation '
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 52.63it/s]

Generated embeddings with shape:(1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_773df510_14',
  'content': 'lOMoARcPSD|285 747 87 \n \n \n\uf0b7 We then get N separate pairs by leaving out a different instance at each \niteration. \n\uf0b7 This is typically used in applications such as medical diagnosis, where \nlabeled data is hard to find. \n\uf0b7  Leave-one-out does not permit stratification. Recently, with computation \ngetting cheaper, it has also become possible to have multiple runs of K-fold \ncross-validation, for example, 10×10- fold, and use average over averages to \nget more reliable error estimates \n5×2 Cross-Validation \n\uf0b7 Dietterich (1998) proposed the 5 × 2 cross-validation, which uses training \nand validation sets of equal size. \n\uf0b7 We divide the dataset X randomly into two parts, X(1) 1and X(2) 1, which gives \nour first pair of training and validation sets, T1 = X(1) 1and V1 = X(2) 1. \n\uf0b7 Then we swap the role of the two halves and get the second pair: T2 = X(2) \n1and V2 = X(1) 1. \n\uf0b7 This is the first fold;

Integration Vectordb Context pipeline with LLM Output

In [65]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])

    return response.content

In [68]:
answer = rag_simple("What is dbms?", rag_retriever, llm)
print(answer)

Retrieving documents for query: 'What is dbms?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.48it/s]

Generated embeddings with shape:(1, 384)
Retrieved 3 documents (after filtering)


DBMS stands for Database Management System. It is a software component that manages and controls the organization's data, comprising the DBMS software itself, application programs, operating system, and network software if used over a network.


#### Enhaced RAG Pipeline Features


In [77]:
# Enhanced RAG Pipeline
def rag_advanced(query,retriever,llm,top_k=5,min_score=0.2,return_context=False):
    """
    RAG Pipeline with extra features:
     - Return answer,sources,confidence score, and optionally full context.
    
    """
    results=retriever.retrieve(query,top_k=top_k,score_threshold=min_score)
    if not results:
        return {'answer':'No Relevant Context Found.','sources':[],'confidence':0.0,'context':''}
    
    # Prepare context and sources
    context="\n\n".join([doc['content'] for doc in results])
    sources=[{
        'source': doc ['metadata'].get('source_file',doc['metadata'].get('source','unknown')),
        'page':doc['metadata'].get('page','unknown'),
        'score':doc['similarity_score'],
        'preview':doc['content'][:300]+'...'
    } for doc in results]
    confidence=max([doc['similarity_score'] for doc in results])

    # Generate answer
    prompt=f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion:{query}\n\nAnswer:"""
    response=llm.invoke([prompt.format(context=context,query=query)])

    output={
        'answer':response.content,
        'sources':sources,
        'confidence':confidence
    }
    if return_context:
        output['context']=context
    return output
# Example Usage
result=rag_advanced("What is database design",rag_retriever,llm,top_k=3,min_score=0.1,return_context=True) 
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])   

Retrieving documents for query: 'What is database design'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 12.42it/s]

Generated embeddings with shape:(1, 384)
Retrieved 3 documents (after filtering)


Answer: Database design is the process of constructing a model of the data used in an enterprise, which involves three stages: 

1. Conceptual Database Design: Constructing a model of the data used in an enterprise, independent of all physical considerations.
2. Logical Database Design: Constructing a model of the data used in an enterprise based on a specific data model, but independent of a particular DBMS and other physical considerations.
3. Physical Database Design: Producing a description of the implementation of the database on secondary storage, including base relations, file organizations, and indexes.

This process is followed by DBMS Selection, which involves choosing an appropriate DBMS to support the database system.
Sources: [{'source': 'Data Modelling.pdf', 'page': 3, 'score': 0.42769527435302734, 'preview': 'combining all parts together. \nDatabase Design: \n\uf0b7 Conceptual Database Design \n\uf0b7 Logical Database Design \n\uf0b7 Physical Database Design \n \nConcept

#### Advanced RAG pipeline: Streaming,Citations,History,Summarization


In [79]:
#Advance RAG pipeline
from typing import List,Dict,Any
import time

class AdvancedRAGPipeline:
    def __init__(self,retriever,llm):
        self.retriever=retriever
        self.llm=llm
        self.history=[]
    def query(self,question:str,top_k:int=5,min_score:float=0.2,stream:bool=False,summarize:bool=False) -> Dict[str,Any]:
        # Retrieve relevant documents
        results=self.retriever.retrieve(question,top_k=top_k,score_threshold=min_score)
        if not results:
            answer="No relevant context found."
            sorces=[]
            context=""
        else:
            context="\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]     
            # Streaming answer simulation
            prompt=f"""Use the following context to answer the questions concisely.\nContext:\n{context}\n\nQuestion:{question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0,len(prompt),80):
                    print(prompt[i:i+80],end='',flush=True)
                    time.sleep(0.05)
                print()
            response=self.llm.invoke([prompt.format(context=context,question=question)])
            answer=response.content     

            # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is database planning", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])   

Retrieving documents for query: 'what is database planning'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 26.32it/s]

Generated embeddings with shape:(1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the questions concisely.
Context:
should identify a particular task that the database system must support. The assumption is that if the database system 
supports the mission objectives, then the mission 

statement should be met. 
Database planning should also include the development of standards that govern how data will be collected, how the 
format should be specified, what documentation will be needed, and how design and implementation should proceed. 
System Definition: 
Describes the scope and boundaries of the database system and the major user views.

should identify a particular task that the database system must support. The assumption is that if the database system 
supports the mission objectives, then the mission statement should be met. 
Database planning should also include the development of standards that govern how data will be collected, how the 
format should be specified, what documentation will be needed, and how design and implementation should proceed. 
System Definition: 
Describes the scope and boundaries of the database system and the major user views.

should identify a particular task that the database system must support. The assumption is that if the datab